<a href="https://colab.research.google.com/github/ananyanivarthi/profile_parsingg/blob/main/AImocktestapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ** Profile Parsing Model**

In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.2 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 50.6 MB/s eta 0:00:00


In [3]:
!pip install spacy-transformers
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 36.0 MB/s eta 0:00:00
✔ Downlo

In [ ]:
!pip uninstall transformers -y

Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2


In [16]:
import requests

# Function to fetch profile components using their respective URLs
def fetch_component_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Ensure that we handle any bad requests
        return response.json()       # Return the JSON response if successful
    except requests.RequestException as e:
        print(f"Failed to fetch data from {url}: {e}")
        return None
def extract_profile_info(email):
    extracted_info = {}

    # Define URL for fetching projects using the user's email
    projects_url = f"https://aitq-api-t4yr74gyka-uc.a.run.app/projects/get?email={email}"

    # Fetch project details from the API
    projects_data = fetch_component_data(projects_url)

    # Check if the API response contains valid project data
    if projects_data:
        extracted_info['Projects'] = "\n".join([
            f"Title: {project['project_title']} \nDescription: {project['description']} \nTechnology: {project['technology_used']} \nLink: {project['project_link']}\n"
            for project in projects_data
        ])
    else:
        extracted_info['Projects'] = "No projects available or failed to fetch data."

    return extracted_info

# Function to save the extracted profile information to a text file
def save_extracted_text(extracted_info, file_path):
    try:
        # Write the extracted project details to a text file
        with open(file_path, 'w', encoding='utf-8') as file:
            for key, value in extracted_info.items():
                file.write(f"{key}:\n{value}\n\n")
        print(f"Extracted profile information saved to '{file_path}'.")
    except IOError as e:
        print(f"Failed to write extracted text to {file_path}: {e}")
def main():
    # Request user input for their email address
    email = input("Enter the user's email address: ").strip()

    # Extract the project details from the API
    extracted_info = extract_profile_info(email)

    # Specify the output file path
    extracted_file_path = "extracted_profile.txt"

    # Save the extracted project details to the file
    save_extracted_text(extracted_info, extracted_file_path)

# Run the main function to start the process
if __name__ == "__main__":
    main()


Enter the user's email address: ananyanivarthi@gmail.com
Extracted profile information saved to 'extracted_profile.txt'.


In [6]:
def read_extracted_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            extracted_text = file.read()
        return extracted_text
    except IOError as e:
        print(f"Failed to read extracted text from {file_path}: {e}")
        return None
import spacy
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Ensure necessary NLTK resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Load the transformer-based SpaCy model
nlp = spacy.load("en_core_web_trf")

def preprocess_text(text):
    # Step 1: Tokenization into sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    # Step 2: Normalize text (convert to lowercase)
    text = text.lower()

    # Step 3: Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Step 4: Stop Words Removal
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in word_tokenize(text) if word not in stop_words]

    # Step 5: Lemmatization using spaCy
    lemmatized_words = []
    for word in filtered_words:
        doc = nlp(word)
        lemmatized_words.append(doc[0].lemma_)

    # Combine the lemmatized words back into a single text
    preprocessed_text = " ".join(lemmatized_words)
    return preprocessed_text
def save_preprocessed_text(preprocessed_text, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(preprocessed_text)
        print(f"Preprocessed text saved to '{file_path}'.")
    except IOError as e:
        print(f"Failed to save preprocessed text to {file_path}: {e}")
def main():
    # Step 1: Read the extracted text from 'extracted_profile.txt'
    extracted_file_path = "/content/extracted_profile.txt"
    extracted_text = read_extracted_text(extracted_file_path)

    if extracted_text:
        # Step 2: Preprocess the extracted text
        preprocessed_text = preprocess_text(extracted_text)

        # Step 3: Save the preprocessed text to 'preprocessed_profile.txt'
        preprocessed_file_path = "/content/preprocessed_profile.txt"
        save_preprocessed_text(preprocessed_text, preprocessed_file_path)

# Run the main function
if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execut

Preprocessed text saved to '/content/preprocessed_profile.txt'.


In [17]:
import spacy

# Load the transformer-based SpaCy model
nlp = spacy.load("en_core_web_trf")

# Define the list of skills and technologies
skills_list = [
    "Python", "Machine Learning", "Deep Learning", "Data Science", "Artificial Intelligence", "SQL", "NoSQL", "Java",
    "C++", "HTML", "CSS", "JavaScript", "TensorFlow", "PyTorch", "R", "Pandas", "AWS", "Azure", "AutoCAD", "SolidWorks",
    "SPICE", "MS Project", "Lean Manufacturing", "Excel", "QuickBooks", "SEO", "PPC", "Agile", "Scrum", "Tableau",
    "Google Analytics", "Medical Terminology", "Patient Care Skills", "EMR Systems", "Clinical Trials", "Photoshop",
    "Illustrator", "Writing and Editing", "Social Media Management", "MERN", "React", "Node", "Express", "MongoDB",
    "Critical Thinking", "Problem Solving", "Research Skills", "Analytical Skills", "Communication Skills",
    "Time Management", "Teamwork", "Adaptability", "Creativity", "Organization Skills", "Information Literacy",
    "Study Skills", "Attention to Detail", "Scientific Method", "Statistical Analysis", "Computational Thinking",
    "Algebra", "Geometry", "Calculus", "Probability", "Qualitative Research", "Quantitative Research",
    "Historical Analysis", "Literary Criticism", "Philosophical Reasoning", "Market Research", "Entrepreneurship",
    "Clinical Skills", "Public Health", "Artistic Skills", "Design Principles", "Technical Drawing", "Systems Design",
    "Language Proficiency", "Cybersecurity Fundamentals", "Network Management", "Cloud Computing Skills","l1","l2","regularization"
]

# Convert skills to lowercase for matching
skills_set = set(skill.lower() for skill in skills_list)

# Function to perform NER and classify entities
def extract_and_classify_entities(text):
    entities = []

    # Process the entire text
    doc = nlp(text)

    # Iterate through the named entities
    for ent in doc.ents:
        entity_text = ent.text.strip().lower()
        if entity_text in skills_set:
            entities.append((ent.text, "SKILL"))
        else:
            entities.append((ent.text, ent.label_))

    # Handle any remaining words not recognized as entities
    words = text.split()
    for word in words:
        word = word.strip().lower()
        if word in skills_set and not any(ent[0].lower() == word for ent in entities):
            entities.append((word, "SKILL"))

    return entities

# Save the extracted and classified entities to a file
def save_entities_to_file(entities, file_path):
    try:
        with open(file_path, 'w') as output_file:
            for entity, label in entities:
                output_file.write(f"{entity}: {label}\n")
        print(f"Extracted and classified entities saved to '{file_path}'.")
    except IOError as e:
        print(f"Failed to write entities to {file_path}: {e}")

# Main function to perform NER and save entities
def main():
    preprocessed_file_path = "/content/preprocessed_profile.txt"

    try:
        with open(preprocessed_file_path, 'r', encoding='utf-8') as file:
            preprocessed_text = file.read()
    except IOError as e:
        print(f"Failed to read preprocessed text from {preprocessed_file_path}: {e}")
        return

    entities = extract_and_classify_entities(preprocessed_text)
    entities_file_path = "/content/extracted_entities.txt"
    save_entities_to_file(entities, entities_file_path)

# Run the main function
if __name__ == "__main__":
    main()


Extracted and classified entities saved to '/content/extracted_entities.txt'.
